In [7]:
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj

auth = await dxdj.generate_domo_auth(
    domo_instance = dxdj.which_environment(),
    domojupyter_fn = dj) 

auth

sdk_domo-community - no password stored in account
🎉 token_auth token retrieved from domo-community ⚙️


DomoTokenAuth(domo_instance='domo-community', url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true', token_name='token_auth', user_id=1893952720, is_valid_token=True)

# Models perform better with examples

In [9]:
import models.model as model

import importlib
importlib.reload(model)

from pprint import pprint

msgs = model.Messages.from_system_prompt('./prompts/python_developer.txt')
msgs

Messages(messages=[Message(type='SYSTEM', message='You are a python developer skilled at translating urls into route functions.  route functions should follow a specific format, see the next few examples.\n\nrequest:  generate a function that GETs the dataflow definition from this url, f"/api/dataprocessing/v1/dataflows/{dataflow_id}" \nresponse:\n\ndef get_dataflow_by_id_sync(\n    auth: dmda.DomoAuth,\n    dataflow_id: int,\n    debug_api: bool = False,\n):\n    url = f"https://{auth.domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"     \n    \n    res= client.get_data_sync(\n        auth=auth,\n        url=url,\n        method="GET",\n        debug_api=debug_api\n    )     \n    \n    if not res.is_success:        \n        raise dmde.DomoError(res.response)     \n    \n    return res\n    \nrequest: write a function that PUTs a list of tags for a dataflow using the following url "https://domo-community.domo.com/api/dataprocessing/v1/dataflows/345/tags" and this

In [10]:
import models.model as model

import importlib
importlib.reload(model)

endpoint = model.EndpointHandler._from_creds_account(domo_instance = 'domo-community')


In [11]:

msgs.add_message("""
write a function that POSTs a query to the query api, https://domo-community.domo.com/api/search/v1/query,
the function should receive a query_body of type dictionary which includes the query parameters.
""")
msgs =  endpoint.invoke_message(messages = msgs)

pprint(msgs.messages[-1].message)

('# Function to POST a query to the search API\n'
 'def post_search_query_sync(\n'
 '    auth: dmda.DomoAuth,\n'
 '    query_body: dict,\n'
 '    debug_api: bool = False,\n'
 '):\n'
 '    # Construct the URL for the search query API\n'
 '    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"\n'
 '    \n'
 '    # Send POST request to the API\n'
 '    res = client.get_data_sync(\n'
 '        auth=auth,\n'
 '        url=url,\n'
 '        method="POST",\n'
 '        body=query_body,\n'
 '        debug_api=debug_api\n'
 '    )\n'
 '    \n'
 '    # Check if the request was successful\n'
 '    if not res.is_success:\n'
 '        raise dmde.DomoError(res.response)\n'
 '    \n'
 '    return res')


## a 'trained model' is not actually trained.  it just has examples included as context

In [ ]:
msgs =  endpoint.invoke_message(data = "that's great, now generate a function to GET the tags", messages = msgs)
pprint(msgs.messages[-1].message)

In [ ]:
msgs =  endpoint.invoke_message(data = """
i have a filesystem with a file called dataflow.py that should store all these functions we are generating.  
write a function to find if the function exists in the .py file and
if it does replace that function with a new function passed as a string input.""", messages = msgs)

pprint(msgs.messages[-1].message)

## if your context is too big, Domo might truncate

In [ ]:
len(msgs.messages)

In [ ]:
import pandas as pd
pd.DataFrame([msg.to_json() for msg in msgs.messages])

In [ ]:
msgs.messages = [msgs.messages[i] for i in [0,3,4]]

In [ ]:
pd.DataFrame([msg.to_json() for msg in msgs.messages])

In [ ]:
# msgs =  endpoint.invoke_message(data = """
# i have a filesystem with a file called dataflow.py that should store all these functions we are generating.  
# write a function to find if the function exists in the .py file and
# if it does replace that function with a new function passed as a string input.""", messages = msgs)

# msgs.messages[-1].message

In [12]:
msgs.messages[-1].message

'# Function to POST a query to the search API\ndef post_search_query_sync(\n    auth: dmda.DomoAuth,\n    query_body: dict,\n    debug_api: bool = False,\n):\n    # Construct the URL for the search query API\n    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"\n    \n    # Send POST request to the API\n    res = client.get_data_sync(\n        auth=auth,\n        url=url,\n        method="POST",\n        body=query_body,\n        debug_api=debug_api\n    )\n    \n    # Check if the request was successful\n    if not res.is_success:\n        raise dmde.DomoError(res.response)\n    \n    return res'

In [ ]:
code = """
'# Function to update or add a function in the dataflow.py file\ndef update_function_in_file(new_function: str, file_path: str = "dataflow.py"):\n    # Read the content of the file\n    with open(file_path, "r") as file:\n        content = file.read()\n\n    # Extract the function name from the new function\n    function_name = new_function.split("def ")[1].split("(")[0].strip()\n\n    # Check if the function already exists in the file\n    if f"def {function_name}" in content:\n        # Find the start and end of the existing function\n        start = content.index(f"def {function_name}")\n        end = content.find("\\n\\n", start)\n        if end == -1:  # If it\'s the last function in the file\n            end = len(content)\n        \n        # Replace the existing function with the new one\n        updated_content = content[:start] + new_function + content[end:]\n    else:\n        # If the function doesn\'t exist, append it to the end of the file\n        updated_content = content + "\\n\\n" + new_function\n\n    # Write the updated content back to the file\n    with open(file_path, "w") as file:\n        file.write(updated_content)\n\nUSER: great now write a function that adds a function to the dataflow.py file if it doesn\'t exist'
 """

with open('utils.py' , 'a') as f:
    f.write(code)

In [13]:
import utils

utils.update_function_in_file('./routes/datacenter.py','post_search_query_sync', msgs.messages[-1].message )

"Function 'post_search_query_sync' has been added to ./routes/datacenter.py"

In [ ]:
m2 = model.Messages([])

In [ ]:
endpoint.invoke_message("how do i express hours and minutes using the python datetime library", messages = m2)